<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto </font></h1>

<h2 ><font size = 3>1. Preparation of the data set</h2>

1a. Download all the dependencies that we will need.

In [3]:
import requests
from bs4 import BeautifulSoup
import csv

import pandas as pd
import numpy as np

1b. Download the data from the Wikipedia site

In [5]:
import requests #import the data 
postal_codes = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
print('Data downloaded!')

Data downloaded!


In [6]:
ps_soup = BeautifulSoup(postal_codes, 'lxml')#get the html

In [25]:
table = ps_soup.find('table')#get the table in the html

table_rows = table.find_all('tr')
table_rows = table_rows[1:len(table_rows)]

l = []

for tr in table_rows:    
    td = tr.find_all('td')    
    rows = [tr.text for tr in td]
    for i,row in enumerate(rows): rows[i] = str(rows[i]).replace("\n","")
    l.append(rows)
    
df = pd.DataFrame(l, columns=["Postalcode", "Borough", "Neighborhood"])
df.head()


,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


1b. Clean the data and create the table

In [27]:
#Data Cleaning based on the following instructions
# - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
# - More than one neighborhood can exist in one postal code area. 
#   Multiple rows will be combined into one row with the neighborhoods separated with a comma
# - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

for tr in table.find_all('tr'):
    i=0
    row=[]
    discard = False
    postalCode = None
    borough = None
    neighbourhood = None
    for td in tr.find_all('td'):
        if i==0:
            postalCode = td.text.rstrip()
        elif i==1:
            if (td.text.rstrip() == 'Not assigned'):
                discard = True
            else:
                borough = td.text.rstrip()
        else:
            i=0
            if (td.text.rstrip() == 'Not assigned'):
                neighbourhood = borough
            else:
                neighbourhood = td.text.rstrip()
        i=i+1
    if (discard == False):
        row.append(postalCode)
        row.append(borough)
        row.append(neighbourhood)
        csv_writer.writerow(row)



In [28]:
df_merge.shape


(103, 3)

<h2 ><font size = 3>2. Get the latitude and the longitude coordinates of each neighborhood</h2>

In [38]:
#The use of geocoder didn't produce any results thus the coordinates of each neighborhood will be taken from the csv file
df_coo=pd.read_csv('http://cocl.us/Geospatial_data')

In [39]:
df_coo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [40]:
df_coo.shape


(103, 3)

In [41]:
#append the coordinates table into the neighborhood table (part 1)
df_complete = df_merge.join(df_coo.set_index('Postal Code'), on='Postalcode')
df_complete.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
